In [1]:
#　整形したデータファイル（CSV）の計算コード。ここでは化学シフト差と強度比を求める。

In [2]:
import os
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import numpy as np

In [3]:
# --- 3. 化学シフト差を計算する関数 (Data Heightの強度比計算を追加) ---
def calculate_chemical_shift_difference(file1_path, file2_path, output_filepath):
    """
    2つのCSVファイルを読み込み、それぞれのw1とw2の化学シフトの差、
    およびData Heightの強度比を計算し、結果を新しいCSVファイルとして保存します。
    NMR複合シフト差の計算に 15N スケーリングファクター 0.2 を使用します。
    結合時にマッチしなかった行数も表示します。
    """
    if not os.path.exists(file1_path):
        print(f"エラー: 最初のファイル '{file1_path}' が存在しません。")
        return
    if not os.path.exists(file2_path):
        print(f"エラー: 2番目のファイル '{file2_path}' が存在しません。")
        return

    output_dir = os.path.dirname(output_filepath)
    if output_dir and not os.path.exists(output_dir):
        print(f"出力ディレクトリ '{output_dir}' が存在しないため、作成します。")
        os.makedirs(output_dir)
    elif not output_dir:
        output_dir = os.getcwd()

    print(f"\n--- 化学シフト差と強度比の計算処理開始 ---")
    print(f"ファイル1: {os.path.basename(file1_path)}")
    print(f"ファイル2: {os.path.basename(file2_path)}")
    print(f"出力ファイルパス: {output_filepath}")

    try:
        df1 = pd.read_csv(file1_path, encoding='utf-8')
        df2 = pd.read_csv(file2_path, encoding='utf-8')

        print(f"\n--- '{os.path.basename(file1_path)}' の最初の数行 ---")
        print(df1.head())
        print(f"\n--- '{os.path.basename(file2_path)}' の最初の数行 ---")
        print(df2.head())

        # 'Data Height' 列を required_cols に追加
        required_cols = ['Assignment', 'w1', 'w2', 'Data Height'] 
        
        for col in required_cols:
            if col not in df1.columns:
                print(f"エラー: '{os.path.basename(file1_path)}' に必要な列 '{col}' が見つかりません。")
                return
            if col not in df2.columns:
                print(f"エラー: '{os.path.basename(file2_path)}' に必要な列 '{col}' が見つかりません。")
                return

        df1_selected = df1[required_cols].copy()
        df2_selected = df2[required_cols].copy()

        initial_rows_df1 = len(df1_selected)
        initial_rows_df2 = len(df2_selected)
        print(f"\nファイル1 ('{os.path.basename(file1_path)}') の初期行数: {initial_rows_df1}")
        print(f"ファイル2 ('{os.path.basename(file2_path)}') の初期行数: {initial_rows_df2}")

        merged_df = pd.merge(df1_selected, df2_selected, on='Assignment', suffixes=('_file1', '_file2'))

        merged_rows = len(merged_df)
        print(f"結合後のデータフレームの行数: {merged_rows}")

        outer_merged = pd.merge(df1_selected, df2_selected, on='Assignment', how='outer', indicator=True)
        left_only_count = len(outer_merged[outer_merged['_merge'] == 'left_only'])
        right_only_count = len(outer_merged[outer_merged['_merge'] == 'right_only'])

        print(f"結合時にファイル1から失われた（マッチしなかった）Assignmentの数: {left_only_count}")
        print(f"結合時にファイル2から失われた（マッチしなかった）Assignmentの数: {right_only_count}")


        if merged_df.empty:
            print("エラー: 'Assignment' 列でのマッチングが見つかりませんでした。ファイルの内容を確認してください。")
            return

        print("\n--- 結合後のデータフレームの最初の数行 ---")
        print(merged_df.head())

        merged_df['w1_diff'] = merged_df['w1_file1'] - merged_df['w1_file2']
        merged_df['w2_diff'] = merged_df['w2_file1'] - merged_df['w2_file2']
        
        # 化学シフト差の計算
        scaling_factor_15N = 0.2
        merged_df['Chemical_Shift_Difference'] = np.sqrt(
            np.square(merged_df['w1_diff'] * scaling_factor_15N) + 
            np.square(merged_df['w2_diff'])
        )

        # 強度比の計算
        # Data Height_file2 が 0 になる場合に備えて、ゼロ除算を避ける処理を追加
        # np.where を使用して、Data Height_file2 が 0 の場合は NaN (Not a Number) にする
        merged_df['Intensity_Ratio'] = np.where(
            merged_df['Data Height_file2'] != 0,
            merged_df['Data Height_file1'] / merged_df['Data Height_file2'],
            np.nan # ゼロ除算の場合はNaN
        )
        print(f"\n--- 強度比計算後のデータフレームの最初の数行 ---")
        print(merged_df.head())


        print("\n--- 計算結果（差分列と強度比追加後）のデータフレームの最初の数行 ---")
        print(merged_df.head())

        merged_df.to_csv(output_filepath, index=False, encoding='utf-8')
        print(f"\n計算結果が '{output_filepath}' に正常に保存されました。")

    except Exception as e:
        print(f"ファイルの読み込み、結合または計算中にエラーが発生しました: {e}")

In [4]:
# --- メイン実行ブロック ---
if __name__ == "__main__":
    root = tk.Tk()
    root.withdraw()

    print("--- 2つの CSV ファイル間の化学シフト差と強度比計算を開始します ---")

    print("最初のNMRデータCSVファイルを選択してください。")
    file1_path = filedialog.askopenfilename(title="最初のNMRデータCSVファイルを選択してください", filetypes=[("CSV files", "*.csv"), ("All files", "*.*")])
    if file1_path:
        print("2番目のNMRデータCSVファイルを選択してください。")
        file2_path = filedialog.askopenfilename(title="2番目のNMRデータCSVファイルを選択してください", filetypes=[("CSV files", "*.csv"), ("All files", "*.*")])
        if file2_path:
            # ユーザーに保存するファイル名とパスを完全に指定させる
            output_filepath = filedialog.asksaveasfilename(
                title="計算結果を保存するファイル名を指定してください",
                defaultextension=".csv", # デフォルトの拡張子
                filetypes=[("CSV files", "*.csv"), ("All files", "*.*")]
            )

            if output_filepath:
                # ユーザーが指定したフルパスをそのまま関数に渡す
                calculate_chemical_shift_difference(file1_path, file2_path, output_filepath)
            else:
                print("出力ファイル名が指定されませんでした。処理を終了します。")
        else:
            print("2番目のファイルが選択されませんでした。処理を終了します。")
    else:
        print("最初のファイルが選択されませんでした。処理を終了します。")

    print("\n--- スクリプト終了 ---")

--- 2つの CSV ファイル間の化学シフト差と強度比計算を開始します ---
最初のNMRデータCSVファイルを選択してください。
2番目のNMRデータCSVファイルを選択してください。

--- 化学シフト差と強度比の計算処理開始 ---
ファイル1: GDP_noHis_100uM_modified.csv
ファイル2: GMPPNP_noHis_100uM_modified.csv
出力ファイルパス: C:/JupyterNotebook/NMR_Folder/CSV_OriginalFolder/caliculatedCSV/100cal/100_GMPPNPvsGDP.csv

--- 'GDP_noHis_100uM_modified.csv' の最初の数行 ---
  Assignment       w1     w2  Data Height
0      T2N-H  122.240  8.517       377924
1      E3N-H  126.853  8.304       352773
2      Y4N-H  121.801  8.757       257450
3      K5N-H  124.585  9.137       295339
4      L6N-H  126.845  9.562       221291

--- 'GMPPNP_noHis_100uM_modified.csv' の最初の数行 ---
  Assignment       w1     w2  Data Height
0      T2N-H  122.098  8.519      1658692
1      E3N-H  126.709  8.314      1633204
2      Y4N-H  122.011  8.723       807546
3      K5N-H  123.780  9.121      1147323
4      L6N-H  126.387  9.238       881104

ファイル1 ('GDP_noHis_100uM_modified.csv') の初期行数: 162
ファイル2 ('GMPPNP_noHis_100uM_modified.csv') の初期行数: 